## 1: ライブラリのインポート

In [30]:
import sqlite3
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, brier_score_loss, classification_report
import numpy as np
from typing import Optional
import os
# 出力幅を広げる設定
pd.set_option('display.width', 1000)  # 任意の幅に設定（例: 200）
pd.set_option('display.max_columns', None)  # 全ての列を表示
#pd.set_option('display.expand_frame_repr', False)  # 横に広げて表示（改行しない）

## 2: データ読み込み・前処理関数の定義 (load_and_process_pbp)

In [31]:
def load_and_process_pbp(db_path: str, limit_rows: Optional[int] = None) -> Optional[pd.DataFrame]:
    """
    SQLiteデータベースからプレイバイプレイデータを読み込み、データ型を処理し、
    ホームチームの勝敗情報を計算して元のデータに結合します。
    (省略... 関数の内容は元のスクリプトと同じ)
    """
    print(f"--- Starting data loading and processing ---")
    print(f"Database path: {db_path}")
    if limit_rows:
        print(f"Row limit: {limit_rows}")
    else:
        print("Row limit: None (loading all rows)")

    try:
        # 1. SQLiteデータベースへの接続
        print("Connecting to database...")
        conn = sqlite3.connect(db_path)
        print("Database connection successful.")

        # 2. SQLクエリの構築
        query = """
        SELECT
            game_id, eventnum, eventmsgtype, eventmsgactiontype, 
            period, pctimestring,
            homedescription, neutraldescription, visitordescription,
            score, scoremargin
        FROM
            play_by_play
        """
        if limit_rows:
            query += f" LIMIT {limit_rows};"
        else:
            query += ";"

        print("Executing SQL query...")
        # 3. クエリ実行とDataFrameへの読み込み
        df = pd.read_sql_query(query, conn)
        print(f"Successfully loaded {len(df)} rows into DataFrame.")

        # 4. データベース接続を閉じる
        conn.close()
        print("Database connection closed.")

        # 5. データ型の確認と変換
        print("Processing data types...")
        df['game_id'] = df['game_id'].astype(str)
        df['pctimestring'] = df['pctimestring'].astype(str)
        df['score'] = df['score'].astype(str)
        df['scoremargin'] = df['scoremargin'].astype(str)
        df['eventnum'] = pd.to_numeric(df['eventnum'], errors='coerce')
        df['eventmsgtype'] = pd.to_numeric(df['eventmsgtype'], errors='coerce')
        df['eventmsgactiontype'] = pd.to_numeric(df['eventmsgactiontype'], errors='coerce') 
        df['period'] = pd.to_numeric(df['period'], errors='coerce')
        desc_cols = ['homedescription', 'neutraldescription', 'visitordescription']
        for col in desc_cols:
            df[col] = df[col].fillna('')
        print("Data type processing complete.")

        # --- 最終スコアの取得と勝敗判定 ---
        print("\nAttempting to determine game outcomes...")
        game_outcomes = pd.Series(dtype=int) # 空のSeriesを初期化

        end_game_events = df[(df['eventmsgtype'] == 13) & (df['score'].str.contains(' - ', na=False))].copy()

        if not end_game_events.empty:
            end_game_events = end_game_events.dropna(subset=['period'])
            if not end_game_events.empty:
                end_game_events['period'] = end_game_events['period'].astype(int)
                final_events = end_game_events.sort_values('period').groupby('game_id').last()

                if 'score' in final_events.columns:
                    scores_split = final_events['score'].str.split(' - ', expand=True)
                    scores_split.columns = ['home_score', 'visitor_score']
                    scores_split['home_score'] = pd.to_numeric(scores_split['home_score'], errors='coerce')
                    scores_split['visitor_score'] = pd.to_numeric(scores_split['visitor_score'], errors='coerce')
                    scores_split = scores_split.dropna(subset=['home_score', 'visitor_score'])

                    if not scores_split.empty:
                        scores_split['home_win'] = (scores_split['home_score'] > scores_split['visitor_score']).astype(int)
                        game_outcomes = scores_split['home_win']
                        print(f"Determined outcomes for {len(game_outcomes)} games.")
                        if limit_rows:
                            print("[Warning] Game outcomes may be incomplete due to the row limit.")
                    # else: (No need for print here in a function, caller can check)
                # else:
            # else:
        # else:

        print("Merging game outcomes back to the main DataFrame...")
        if not game_outcomes.empty:
             df_with_outcome = df.merge(game_outcomes.rename('home_win'), on='game_id', how='left')
        else:
             print("No game outcomes determined, adding 'home_win' column with NaN.")
             df['home_win'] = pd.NA
             df_with_outcome = df

        print("--- Data loading and processing finished ---")
        return df_with_outcome

    except sqlite3.Error as e:
        print(f"\n--- Database Error ---")
        print(f"An error occurred while interacting with the database: {e}")
        return None
    except FileNotFoundError:
        print(f"\n--- File Not Found Error ---")
        print(f"Error: The database file was not found at the specified path: {db_path}")
        return None
    except Exception as e:
        print(f"\n--- An Unexpected Error Occurred ---")
        print(f"Error details: {e}")
        return None

## 3: 特徴量エンジニアリング用関数の定義

In [32]:
def parse_time_to_seconds(time_str):
    """ 'MM:SS' 形式の文字列を秒に変換 """
    if isinstance(time_str, str) and ':' in time_str:
        try:
            minutes, seconds = map(int, time_str.split(':'))
            return minutes * 60 + seconds
        except ValueError:
            return None # パースエラー
    return None

def calculate_seconds_elapsed(row):
    """ 試合開始からの経過秒数を計算 """
    period = row['period']
    pctimestring = row['pctimestring']

    if pd.isna(period) or period < 1:
        return None

    seconds_in_period = parse_time_to_seconds(pctimestring)
    if seconds_in_period is None:
        return None

    seconds_per_period = 720 if period <= 4 else 300
    seconds_elapsed_in_current_period = seconds_per_period - seconds_in_period

    if seconds_elapsed_in_current_period < 0 or seconds_elapsed_in_current_period > seconds_per_period:
         return None

    if period <= 4:
        total_seconds_elapsed = (period - 1) * 720 + seconds_elapsed_in_current_period
    else:
        total_seconds_elapsed = 4 * 720 + (period - 5) * 300 + seconds_elapsed_in_current_period
    return total_seconds_elapsed

def process_score_margin(margin_str):
    """ scoremargin を数値に変換 ('TIE' -> 0) """
    if margin_str == 'TIE':
        return 0
    elif pd.isna(margin_str) or margin_str == '':
         return None
    else:
        try:
            return int(str(margin_str).replace('+', ''))
        except ValueError:
            return None

## 4 設定とデータ読み込みの実行

In [33]:
#研究室PC用path
db_file = r'C:\Users\amilu\Projects\vsCodeFile\PBL\nba.sqlite'
#ノートPC用path
#db_file = 'C:\Workspace\PBL\\nba.sqlite'

limit_rows = 45616
# limit_rows = None # 全データの場合

df_processed = load_and_process_pbp(db_file, limit_rows=limit_rows)

# 読み込み結果の確認
if df_processed is not None:
    print("\nShape of loaded data:", df_processed.shape)
    df_processed.head() # 先頭数行を表示
else:
    print("Data loading failed.")

--- Starting data loading and processing ---
Database path: C:\Users\amilu\Projects\vsCodeFile\PBL\nba.sqlite
Row limit: 45616
Connecting to database...
Database connection successful.
Executing SQL query...
Successfully loaded 45616 rows into DataFrame.
Database connection closed.
Processing data types...
Data type processing complete.

Attempting to determine game outcomes...
Determined outcomes for 99 games.
[Warning] Game outcomes may be incomplete due to the row limit.
Merging game outcomes back to the main DataFrame...
--- Data loading and processing finished ---

Shape of loaded data: (45616, 12)


## 5: 特徴量エンジニアリングの実行

In [34]:
if df_processed is not None:
    print("\n--- Feature Engineering ---")

    print("Calculating total seconds elapsed...")
    df_processed['seconds_elapsed'] = df_processed.apply(calculate_seconds_elapsed, axis=1)

    print("Processing score margin...")
    # Step 1: 'TIE' や数値変換可能なものを数値に変換 (NoneはNoneのまま)
    df_processed['numeric_score_margin'] = df_processed['scoremargin'].apply(process_score_margin)

    # Step 2: game_id ごとに並べ替え、欠損値 (None) を直前の有効な値で前方補完 (ffill)
    print("Forward filling missing 'numeric_score_margin' within each game...")
    # game_id と eventnum でソートすることが重要
    df_processed = df_processed.sort_values(by=['game_id', 'eventnum'])
    df_processed['numeric_score_margin'] = df_processed.groupby('game_id')['numeric_score_margin'].ffill()

    # ffill 後も NaN が残る場合がある（ゲームの最初の数プレイなど、前に有効な値がない場合）
    # これらは後続の dropna で処理されるか、別途 0 などで埋める判断も可能
    # print("NaN count in numeric_score_margin after ffill:", df_processed['numeric_score_margin'].isnull().sum())
    # 2.3 Generate Composite Event ID
    print("Generating composite event ID...")
    # eventmsgtype と eventmsgactiontype が数値であることを確認 (NaNの場合は計算結果もNaNになる)
    # 複合IDの計算前に、これらのカラムの欠損値処理が必要か検討
    # 例: df_processed['eventmsgtype'].fillna(0, inplace=True) # 欠損を0で埋める場合
    #     df_processed['eventmsgactiontype'].fillna(0, inplace=True)
    
    # もし欠損値のまま計算すると、結果がNaNになるため、後続のdropnaで除外されるか、
    # またはfillna(例えば -1 や特定の予約ID) で埋める
    df_processed['composite_event_id'] = (df_processed['eventmsgtype'] * 1000 + df_processed['eventmsgactiontype'])
    # 欠損値から生じたNaNを、例えば不明なID (-1など) で埋める場合
    # df_processed['composite_event_id'] = df_processed['composite_event_id'].fillna(-1).astype(int)
    print("Composite event ID generation complete.")


    # 結果の確認
    print("\nPreview of processed time, score margin, and composite event ID features:")
    print(df_processed[['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'composite_event_id', 'seconds_elapsed', 'numeric_score_margin']].head(15))


--- Feature Engineering ---
Calculating total seconds elapsed...
Processing score margin...
Forward filling missing 'numeric_score_margin' within each game...
Generating composite event ID...
Composite event ID generation complete.

Preview of processed time, score margin, and composite event ID features:
         game_id  eventnum  eventmsgtype  eventmsgactiontype  composite_event_id  seconds_elapsed  numeric_score_margin
2225  0029600001         1            12                   0               12000                0                   NaN
2226  0029600001         2            10                   0               10000                0                   NaN
2227  0029600001         4             1                   5                1005               21                  -2.0
2228  0029600001         5             6                   2                6002               21                  -2.0
2229  0029600001         6             3                  10                3010            

## 6: モデル用データ準備 (フィルタリング)

In [35]:
if df_processed is not None:
    print("\n--- Data Preparation for Modeling ---")
    print("Filtering data for modeling...")
    initial_rows = len(df_processed)
    
    # 欠損値の確認 (フィルタリング前)
    # print("NaN counts before filtering:")
    # print(df_processed[['home_win', 'seconds_elapsed', 'numeric_score_margin', 'period']].isnull().sum())

    model_df = df_processed.dropna(subset=['home_win', 'seconds_elapsed', 'numeric_score_margin', 'period', 'composite_event_id'])
    model_df = model_df[model_df['period'] > 0]
    model_df = model_df[model_df['eventmsgtype'] != 12] # "Start Period" イベントを除外

    filtered_rows = len(model_df)
    print(f"Rows before filtering: {initial_rows}")
    print(f"Rows after filtering invalid/unnecessary entries: {filtered_rows}")
    
    if filtered_rows > 0:
        model_df.head()
    else:
        print("No data left after filtering.")



--- Data Preparation for Modeling ---
Filtering data for modeling...
Rows before filtering: 45616
Rows after filtering invalid/unnecessary entries: 44791


## 7: 特徴量とターゲットの選択、訓練/テスト分割

In [36]:
if 'model_df' in locals() and not model_df.empty:
    # --- One-Hot Encoding for composite_event_id ---
    print("Applying One-Hot Encoding to 'composite_event_id'...")
    # NaNや不明なIDで埋めた場合は、それも1つのカテゴリとして扱われる
    # あまりに多くのカテゴリがある場合は注意が必要 (次元の呪い)
    # ここでは、dropnaでcomposite_event_idがNaNの行は除外されている前提
    
    # One-Hotエンコーディングを実行し、元のDataFrameに結合
    # get_dummiesはNaNを無視するか、専用のカテゴリを作るか選択できる
    # dtype=int で 0/1 の整数型にする
    model_df_encoded = pd.get_dummies(model_df, columns=['composite_event_id'], prefix='event', dtype=int)
    print(f"Shape after One-Hot Encoding: {model_df_encoded.shape}")
    print("New columns created by One-Hot Encoding (first few):")
    print([col for col in model_df_encoded.columns if col.startswith('event_')][:5])


    # --- 特徴量とターゲットの選択 ---
    # 元の特徴量に、One-Hotエンコードされた列を追加
    base_features = ['numeric_score_margin', 'seconds_elapsed']
    one_hot_event_features = [col for col in model_df_encoded.columns if col.startswith('event_')]
    
    features = base_features + one_hot_event_features
    target = 'home_win'

    print(f"\nTotal features for the model: {len(features)}")
    # print("Selected features:", features) # 長くなるのでコメントアウト

    X = model_df_encoded[features]
    y = model_df_encoded[target].astype(int) # model_df_encoded を使用

    print("\nSplitting data into training and testing sets (game-aware)...")
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    
    if model_df_encoded['game_id'].nunique() < 2: # model_df_encoded を使用
        print("Warning: Not enough unique games for GroupShuffleSplit...")
        # (代替処理は省略、元のコードと同様)
        if len(X) > 1:
             train_idx, test_idx = train_test_split(range(len(X)), test_size=0.2, random_state=42, stratify=y if y.nunique() > 1 else None)
        else:
            train_idx, test_idx = range(len(X)), []
            print("Too few samples for splitting. Using all as training data.")
    else:
        train_idx, test_idx = next(gss.split(X, y, groups=model_df_encoded['game_id'])) # model_df_encoded を使用

    if len(train_idx) > 0 and len(test_idx) > 0:
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        print(f"Training set size: {len(X_train)}")
        print(f"Testing set size: {len(X_test)}")
        
        # one-hotエンコード後のX_trainの最初の数行を表示して確認
        # print("\nSample of X_train after one-hot encoding (first 5 rows, first 10 columns):")
        # print(X_train.iloc[:5, :10])

    elif len(train_idx) > 0:
        print(f"Only training data available. Training set size: {len(X_train)}")
        X_test, y_test = pd.DataFrame(columns=X.columns), pd.Series(dtype=y.dtype)
    else:
        print("Could not create valid train/test splits.")
else:
    print("Skipping feature selection and train/test split as model_df is not available or empty.")

Applying One-Hot Encoding to 'composite_event_id'...
Shape after One-Hot Encoding: (44791, 92)
New columns created by One-Hot Encoding (first few):
['event_1001', 'event_1002', 'event_1003', 'event_1004', 'event_1005']

Total features for the model: 80

Splitting data into training and testing sets (game-aware)...
Training set size: 35629
Testing set size: 9162


## 8: モデル訓練 (スケーリングとロジスティック回帰)

In [37]:
if 'X_train' in locals() and not X_train.empty:
    print("\n--- Model Training ---")
    print("Scaling features...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # テストデータがある場合のみスケーリング
    if not X_test.empty:
        X_test_scaled = scaler.transform(X_test)
    else:
        X_test_scaled = np.array([]) # 空の配列

    print("Training Logistic Regression model...")
    model = LogisticRegression(random_state=42)
    model.fit(X_train_scaled, y_train)
    print("Model training complete.")
else:
    print("Skipping model training as training data is not available.")


--- Model Training ---
Scaling features...
Training Logistic Regression model...
Model training complete.


##  9: 予測と評価

In [38]:
if 'model' in locals() and 'X_test_scaled' in locals() and X_test_scaled.shape[0] > 0: # テストデータがあるか確認
    print("\n--- Prediction and Evaluation ---")
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    y_pred_class = model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred_class)
    auc = roc_auc_score(y_test, y_pred_proba)
    logloss = log_loss(y_test, y_pred_proba)
    brier = brier_score_loss(y_test, y_pred_proba)

    print(f"Model Evaluation Results:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  ROC AUC:  {auc:.4f}")
    print(f"  Log Loss: {logloss:.4f}")
    print(f"  Brier Score: {brier:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_class))

    # テストデータに予測結果を結合
    # model_df_encoded から test_idx を使ってスライスする
    model_df_test = model_df_encoded.iloc[test_idx].copy()
    model_df_test['win_probability_pred'] = y_pred_proba

    # numeric_score_margin の正負反転
    if 'numeric_score_margin' in model_df_test.columns:
        print("\nReversing the sign of 'numeric_score_margin' in model_df_test...")
        model_df_test['numeric_score_margin_invation'] = model_df_test['numeric_score_margin'] * -1
        print("Sign reversal of 'numeric_score_margin_invation' complete.")
        print(model_df_test[['scoremargin', 'numeric_score_margin', 'numeric_score_margin_invation']].head())
    else:
        print("\nWarning: 'numeric_score_margin' column not found in model_df_test.")


    print("\nFirst 5 rows of test data with predicted win probability:")
    # 表示するカラムを調整
    display_cols = ['game_id', 'eventnum', 'period', 'pctimestring', 'score', 'scoremargin', 'numeric_score_margin', 'seconds_elapsed', 'home_win', 'win_probability_pred']
    if 'numeric_score_margin_invation' in model_df_test.columns:
         display_cols.append('numeric_score_margin_invation')
    print(model_df_test[display_cols].head())

else:
    print("Skipping prediction and evaluation as model or test data is not available.")


--- Prediction and Evaluation ---
Model Evaluation Results:
  Accuracy: 0.6644
  ROC AUC:  0.7200
  Log Loss: 0.5917
  Brier Score: 0.2081

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.70      0.71      5304
           1       0.60      0.62      0.61      3858

    accuracy                           0.66      9162
   macro avg       0.66      0.66      0.66      9162
weighted avg       0.67      0.66      0.66      9162


Reversing the sign of 'numeric_score_margin' in model_df_test...
Sign reversal of 'numeric_score_margin_invation' complete.
     scoremargin  numeric_score_margin  numeric_score_margin_invation
2227          -2                  -2.0                            2.0
2228        None                  -2.0                            2.0
2229          -3                  -3.0                            3.0
2230          -1                  -1.0                            1.0
2233           1                  

## 10: CSV出力

In [39]:
if 'model_df_test' in locals() and isinstance(model_df_test, pd.DataFrame) and not model_df_test.empty:
    output_csv_path = 'win_probability_predictions_ipynb.csv' # Notebookからの出力とわかるように名前変更
    print(f"\n--- Exporting Test Predictions to CSV ---")
    try:
        model_df_test.to_csv(output_csv_path, index=False, encoding='utf-8')
        print(f"Successfully exported test predictions to: {output_csv_path}")
    except Exception as e:
        print(f"Error exporting to CSV: {e}")
else:
    print("\nSkipping CSV export because 'model_df_test' was not generated, is empty, or is not a DataFrame.")

print("\n--- Win probability prediction notebook execution finished (potentially) ---")


--- Exporting Test Predictions to CSV ---
Successfully exported test predictions to: win_probability_predictions_ipynb.csv

--- Win probability prediction notebook execution finished (potentially) ---
